# Data Loading and Cleaning

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from typing import Dict, List, Optional, Set, Tuple, Optional, Union
from datetime import datetime
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz, process
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.inspection import permutation_importance
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [ ]:
# File paths
lb_path = "LB_animal-shelter-intakes-and-outcomes.csv"
sj_path = "combined_SJ_animalshelterdata.csv"
bloomington_path = "Bloomington_Animal_Shelter_Animals.csv"
akc_path = "akc-data-latest.csv"
df_akc = pd.read_csv(akc_path)
df_akc = df_akc.rename(columns={'Unnamed: 0': 'dog_breed_name'})

# Try reading the files, detect separator if needed (assuming comma for now)
df_lb = pd.read_csv(lb_path)
df_sj = pd.read_csv(sj_path)
df_bloom = pd.read_csv(bloomington_path)

# Filter and standardize
df_lb = df_lb[
    df_lb['Animal Type'].astype(str).str.strip().str.lower().isin(['dog', 'cat'])
].copy()

df_sj = df_sj[
    df_sj['AnimalType'].astype(str).str.strip().str.lower().isin(['dog', 'cat'])
].copy()

df_bloom = df_bloom[
    df_bloom['speciesname'].astype(str).str.strip().str.lower().isin(['dog', 'cat'])
].copy()

# Normalize the values to lowercase
df_lb['Animal Type'] = df_lb['Animal Type'].astype(str).str.strip().str.lower()
df_sj['AnimalType'] = df_sj['AnimalType'].astype(str).str.strip().str.lower()
df_bloom['speciesname'] = df_bloom['speciesname'].astype(str).str.strip().str.lower()


# Check their shapes and first few columns
lb_info = (df_lb.shape, list(df_lb.columns))
sj_info = (df_sj.shape, list(df_sj.columns))
bloom_info = (df_bloom.shape, list(df_bloom.columns))

lb_info, sj_info, bloom_info

(((17648, 23),
  ['Animal ID',
   'Animal Name',
   'Animal Type',
   'Primary Color',
   'Secondary Color',
   'Sex',
   'DOB',
   'Intake Date',
   'Intake Condition',
   'Intake Type',
   'Intake Subtype',
   'Reason for Intake',
   'Outcome Date',
   'Crossing',
   'Jurisdiction',
   'Outcome Type',
   'Outcome Subtype',
   'latitude',
   'longitude',
   'intake_is_dead',
   'outcome_is_dead',
   'was_outcome_alive',
   'geopoint']),
 ((107685, 21),
  ['AnimalID',
   'AnimalName',
   'AnimalType',
   'PrimaryColor',
   'SecondaryColor',
   'PrimaryBreed',
   'Sex',
   'DOB',
   'Age',
   'IntakeDate',
   'IntakeCondition',
   'IntakeType',
   'IntakeSubtype',
   'IntakeReason',
   'OutcomeDate',
   'OutcomeType',
   'OutcomeSubtype',
   'OutcomeCondition',
   'Crossing',
   'Jurisdiction',
   'LastUpdate']),
 ((11988, 23),
  ['id',
   'intakedate',
   'intakereason',
   'istransfer',
   'sheltercode',
   'identichipnumber',
   'animalname',
   'breedname',
   'basecolour',
   'spec

In [ ]:
df_lb['Animal Type'].value_counts()

,count
Animal Type,
cat,10833
dog,6815


In [ ]:
df_lb['Outcome Date'] = pd.to_datetime(df_lb['Outcome Date'], errors='coerce')
df_lb['outcome_year'] = df_lb['Outcome Date'].dt.year
outcome_year_counts_lb = df_lb['outcome_year'].value_counts().sort_index()

df_sj['OutcomeDate'] = pd.to_datetime(df_sj['OutcomeDate'], errors='coerce')
df_sj['outcome_year'] = df_sj['OutcomeDate'].dt.year
outcome_year_counts_sj = df_sj['outcome_year'].value_counts().sort_index()

df_bloom['movementdate'] = pd.to_datetime(df_bloom['movementdate'], errors='coerce')
df_bloom['outcome_year'] = df_bloom['movementdate'].dt.year
outcome_year_counts_bloom = df_bloom['outcome_year'].value_counts().sort_index()

print(outcome_year_counts_lb)
print(outcome_year_counts_sj)
print(outcome_year_counts_bloom)

/tmp/ipython-input-6-1714429800.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_bloom['movementdate'] = pd.to_datetime(df_bloom['movementdate'], errors='coerce')


outcome_year
2017.0    2651
2018.0    2433
2019.0    2193
2020.0    1887
2021.0    1573
2022.0    1707
2023.0    2128
2024.0    2224
2025.0     747
Name: count, dtype: int64
outcome_year
2018.0     8987
2019.0    17388
2020.0    12964
2021.0    16284
2022.0    10956
2023.0    15987
2024.0    17930
2025.0     6171
2034.0        1
Name: count, dtype: int64
outcome_year
2013.0       1
2016.0       1
2017.0    3036
2018.0    3807
2019.0    4325
2020.0     404
2021.0      84
2022.0      75
2023.0      63
2024.0      31
2025.0      18
Name: count, dtype: int64


In [ ]:
primary_colors_counts_lb = df_lb['Primary Color'].value_counts()
print(primary_colors_counts_lb.head(30))

primary_colors_counts_sj = df_sj['PrimaryColor'].value_counts()
print(primary_colors_counts_sj.head(30))

Primary Color
BLACK         4781
WHITE         2194
GRAY          1710
BRN TABBY     1709
BROWN         1347
TAN            941
GRAY TABBY     885
ORG TABBY      682
CALICO         403
ORANGE         371
TORTIE         349
BR BRINDLE     223
TRICOLOR       194
CREAM          180
SEAL PT        162
BLUE           158
RED            115
TORTIE DIL      97
LYNX PT         86
FAWN            84
CALICO DIL      77
GOLD            66
BLONDE          66
UNKNOWN         60
CHOCOLATE       59
CRM TABBY       58
CALICO TAB      57
BLK TABBY       54
SNOWSHOE        39
CHOC PT         38
Name: count, dtype: int64
PrimaryColor
BLACK         30665
TABBY-BRN     16092
WHITE          9982
GRAY           9181
TABBY-ORG      6898
TABBY-GRAY     5602
BROWN          5534
TAN            4682
TORTIE-B\O     2333
CALICO-TRI     2063
TORBI-BRN      1336
TRICOLOR       1269
ORANGE         1214
PT-LYNX        1066
BRINDLE-BN     1055
CREAM           936
CALICO-DIL      808
TABBY-BUFF      713
PT-SEAL         7

In [ ]:
secondary_colors_counts_lb = df_lb['Secondary Color'].value_counts()
print(secondary_colors_counts_lb.head(30))

secondary_colors_counts_sj = df_sj['SecondaryColor'].value_counts()
print(secondary_colors_counts_sj.head(30))

Secondary Color
WHITE            5551
BLACK             955
BROWN             790
TAN               540
GRAY              352
BRN TABBY          94
ORANGE             70
CREAM              63
GRAY TABBY         31
CALICO             24
BR BRINDLE         23
RED                22
TRICOLOR           20
SILVER             20
TORTIE             16
BLUE               15
ORG TABBY          14
GOLD               14
BL BRINDLE          5
FAWN                4
YELLOW              4
SABLE               3
FLAME PT            3
BLUE TICK           2
BLK TABBY           2
BUFF                2
BLK SMOKE           1
MARBLED TABBY       1
RED TICK            1
LYNX PT             1
Name: count, dtype: int64
SecondaryColor
WHITE         32180
BLACK          4171
BROWN          2807
TAN            2440
GRAY           1409
TABBY-BRN       387
ORANGE          260
CREAM           240
TABBY-GRAY      130
TABBY-ORG       109
RED              89
BRINDLE-BN       82
OTHER            61
TRICOLOR         49
GOL

In [ ]:
sex_counts_lb = df_lb['Sex'].value_counts()
print(sex_counts_lb.head(30))

sex_count_sj = df_sj['Sex'].value_counts()
print(sex_count_sj.head(30))

Sex
Male        4614
Female      4234
Neutered    3971
Spayed      3415
Unknown     1413
Name: count, dtype: int64
Sex
NEUTERED    33245
SPAYED      32002
UNKNOWN     18126
MALE        13198
FEMALE      11114
Name: count, dtype: int64


In [ ]:
intakecondition_counts_lb = df_lb['Intake Condition'].value_counts()
print(intakecondition_counts_lb.head(30))

intakecondition_counts_sj = df_sj['IntakeCondition'].value_counts()
print(intakecondition_counts_sj.head(30))

Intake Condition
NORMAL                8977
UNDER AGE/WEIGHT      4029
ILL MILD               757
INJURED  SEVERE        668
ILL SEVERE             572
ILL MODERATETE         546
FRACTIOUS              518
INJURED  MILD          432
INJURED  MODERATE      400
FERAL                  225
I/I REPORT             183
AGED                   107
BEHAVIOR  MODERATE      86
WELFARE SEIZURES        53
BEHAVIOR  SEVERE        50
BEHAVIOR  MILD          44
Name: count, dtype: int64
IntakeCondition
NORMAL        35510
HEALTHY       15828
OTHER MED     14865
DEAD          10950
NURSING        9865
MED R          7079
FERAL          3319
MED SEV        2552
INJURED        1898
MED M          1683
SICK           1470
PREGNANT        868
MED EMERG       460
BEH M           378
UNHEALTHY       338
BEH U           253
AGGRESSIVE      162
BEH R           133
FEARFUL          39
AGED             27
MANAGE            4
REHAB             4
Name: count, dtype: int64


In [ ]:
intaketype_counts_lb = df_lb['Intake Type'].value_counts()
print(intaketype_counts_lb.head(30))

intaketype_counts_sj = df_sj['IntakeType'].value_counts()
print(intaketype_counts_sj.head(30))


Intake Type
STRAY                    14652
OWNER SURRENDER           1856
WELFARE SEIZED             404
CONFISCATE                 278
RETURN                     231
QUARANTINE                 112
SAFE KEEP                   67
TRAP, NEUTER, RETURN        43
WILDLIFE                     1
FOSTER                       1
Euthenasia Required          1
Adopted Animal Return        1
Name: count, dtype: int64
IntakeType
STRAY         75914
DISPO REQ     10723
FOSTER         6252
OWNER SUR      4934
S/N CLINIC     3784
CONFISCATE     2435
RETURN         1505
TRANSFER       1380
EUTH REQ        668
DISASTER         37
NEUTER           26
SPAY             24
WILDLIFE          3
Name: count, dtype: int64


In [ ]:
intakesubtype_counts_lb = df_lb['Intake Subtype'].value_counts()
print(intakesubtype_counts_lb.head(30))

intakesubtype_counts_sj = df_sj['IntakeSubtype'].value_counts()
print(intakesubtype_counts_sj.head(30))


Intake Subtype
OTC           10053
FIELD          5847
TRAP            383
POLICE          328
ADOPTION        157
HOSPITAL        126
ABAN FIELD      112
OWNER DIED       96
CRUELTY          63
ABAN SHLTR       42
EVICTION         35
BITE             26
BORN@SHELT       22
AT SHELTER       16
RTF              15
VICIOUS          11
PUB SAFETY       10
EMERGENCY         8
SPCALA            7
RESCUE            5
ABANDON           4
INVESTIGAT        1
Name: count, dtype: int64
IntakeSubtype
OTC           72494
FIELD         14651
MEDVET         1990
FERAL          1633
OTC OWNED      1387
FOUND          1365
ASO             918
POLICE          890
OTHER AGEN      846
SVVS            643
EAC             596
HOSPITAL        188
RESCUE AS       178
COUNTY          150
BITE            134
OWNER DIED      130
CRUELTY         126
OWNED           124
SVACA           117
FIELD OWN       105
RESCUE NON       51
EVICTION         46
HSSV             37
FOSTER           18
STRAY            15
TOWN 

In [ ]:
intakereason_counts_lb = df_lb['Reason for Intake'].value_counts()
print(intakereason_counts_lb.head(30))

intakereason_counts_sj = df_sj['IntakeReason'].value_counts()
print(intakereason_counts_sj.head(30))


Reason for Intake
OWNER PROB    294
MOVE          164
LANDLORD      127
COST           87
NO HOME        82
NO TIME        80
OWNER DIED     63
TOO MANY       62
BITES          59
AGG ANIMAL     58
ILL            54
ALLERGIC       46
OTHER PET      39
AGG PEOPLE     37
ABANDON        25
HYPER          20
INJURED        16
POOR HELTH     15
ESCAPES        14
UNKNOWN        13
NEW BABY        8
CHILD PROB      8
DIVORCE         7
RESPONSIBL      6
HOUSE SOIL      6
ATTENTION       5
NOFRIENDLY      5
DESTRUC IN      5
JUMPS UP        5
AFRAID          4
Name: count, dtype: int64
IntakeReason
IP ADOPT      2429
IP EUTH       1033
LANDLORD       346
DOA            334
MOVE           317
OWNER PROB     220
NO TIME        218
ALLERGIC       165
OTHER PET      146
NO HOME        136
TOO MANY       131
RESPONSIBL     117
BITES           98
COST            89
AGG PEOPLE      70
AGG ANIMAL      68
OWNER DIED      53
HOUSE SOIL      47
HYPER           45
CHILD PROB      37
EUTH ILL        36
ESCA

In [ ]:
outcometype_counts_lb = df_lb['Outcome Type'].value_counts()
print(outcometype_counts_lb.head(30))

outcometype_counts_sj = df_sj['OutcomeType'].value_counts()
print(outcometype_counts_sj.head(30))


Outcome Type
ADOPTION                   4283
RESCUE                     4173
TRANSFER                   2493
EUTHANASIA                 2480
RETURN TO OWNER            2290
SHELTER, NEUTER, RETURN     696
COMMUNITY CAT               347
DIED                        315
FOSTER TO ADOPT             161
TRANSPORT                    89
MISSING                      51
HOMEFIRST                    47
TRAP, NEUTER, RELEASE        37
RETURN TO RESCUE             31
DUPLICATE                    19
DISPOSAL                     13
RETURN TO WILD HABITAT        8
FOSTER                        6
Name: count, dtype: int64
OutcomeType
ADOPTION      22661
RESCUE        20119
TRANSFER      18755
DISPOSAL      10911
RTO           10153
EUTH           7618
FOSTER         6497
RTF            5469
DIED           1863
FOUND EXP       701
FOUND ANIM      660
SPAY            524
NEUTER          418
LOST EXP        146
S/N UNABLE       73
MISSING          51
REQ EUTH         39
Name: count, dtype: int64


In [ ]:
outcomesubtype_count_lb = df_lb['Outcome Subtype'].value_counts()
print(outcomesubtype_count_lb.head(30))

outcomesubtype_count_sj = df_sj['OutcomeSubtype'].value_counts()
print(outcomesubtype_count_sj.head(30))


Outcome Subtype
SPCALA        2619
WEB           2044
WALKIN        2008
ILL SEVERE     951
LITTLELION     821
STRAYCATAL     760
LITTLEPAWS     693
CATPAWS        491
AT VET         449
UNDRAGE/WT     416
FNDANIFOUN     411
PFE/PAWSHP     366
FRE RID HM     358
OTHER RESC     331
INJ SEVERE     325
MICROCHIP      176
SBACC          151
LIVELOVE       146
BEH SEVERE     142
CAPAWS         141
IN KENNEL      134
FRIENDLY       117
ILL MODERA     108
COMMCAT         89
ENROUTE         78
BEH MODERA      75
KITTYBUNGA      62
SPEC EVT        58
SPARKYGANG      55
PHONE           53
Name: count, dtype: int64
Series([], Name: count, dtype: int64)


## Standarize and align columns across datasets

In [ ]:
# Step 1: Standardize and align columns across datasets
def standardize_columns(df, mapping, shelter_name):
    df = df.rename(columns=mapping)
    df['shelter'] = shelter_name
    return df

# Define mappings from each dataset to a common schema
# We'll define a core set of columns that make sense across all datasets
common_columns = {
    'animal_id': 'animal_id',
    'animal_name': 'animal_name',
    'animal_type': 'animal_type',
    'primary_color': 'primary_color',
    'secondary_color': 'secondary_color',
    'primary_breed': 'primary_breed',
    'sex': 'sex',
    'dob': 'dob',
    'intake_date': 'intake_date',
    'intake_condition': 'intake_condition',
    'intake_type': 'intake_type',
    'intake_subtype': 'intake_subtype',
    'intake_reason': 'intake_reason',
    'outcome_date': 'outcome_date',
    'outcome_type': 'outcome_type',
    'outcome_subtype': 'outcome_subtype'
}

# Long Beach mapping
lb_mapping = {
    'Animal ID': 'animal_id',
    'Animal Name': 'animal_name',
    'Animal Type': 'animal_type',
    'Primary Color': 'primary_color',
    'Secondary Color': 'secondary_color',
    'Sex': 'sex',
    'DOB': 'dob',
    'Intake Date': 'intake_date',
    'Intake Condition': 'intake_condition',
    'Intake Type': 'intake_type',
    'Intake Subtype': 'intake_subtype',
    'Reason for Intake': 'intake_reason',
    'Outcome Date': 'outcome_date',
    'Outcome Type': 'outcome_type',
    'Outcome Subtype': 'outcome_subtype'
}

# San Jose mapping
sj_mapping = {
    'AnimalID': 'animal_id',
    'AnimalName': 'animal_name',
    'AnimalType': 'animal_type',
    'PrimaryColor': 'primary_color',
    'SecondaryColor': 'secondary_color',
    'PrimaryBreed': 'primary_breed',
    'Sex': 'sex',
    'DOB': 'dob',
    'IntakeDate': 'intake_date',
    'IntakeCondition': 'intake_condition',
    'IntakeType': 'intake_type',
    'IntakeSubtype': 'intake_subtype',
    'IntakeReason': 'intake_reason',
    'OutcomeDate': 'outcome_date',
    'OutcomeType': 'outcome_type',
    'OutcomeSubtype': 'outcome_subtype'
}

# Bloomington mapping
bloom_mapping = {
    'id': 'animal_id',
    'animalname': 'animal_name',
    'speciesname': 'animal_type',
    'basecolour': 'primary_color',
    'breedname': 'primary_breed',
    'sexname': 'sex',
    'intakedate': 'intake_date',
    'intakereason': 'intake_reason',
    'movementdate': 'outcome_date',
    'movementtype': 'outcome_type',
    'deceaseddate': 'deceased_date',
    'returndate': 'return_date',
    'diedoffshelter': 'diedoffshelter',
    'puttosleep': 'puttosleep',
    'isdoa': 'isdoa',
    'animalage': 'Age'
}

# Apply standardization
df_lb_std = standardize_columns(df_lb, lb_mapping, 'Long Beach')
df_sj_std = standardize_columns(df_sj, sj_mapping, 'San Jose')
df_bloom_std = standardize_columns(df_bloom, bloom_mapping, 'Bloomington')


# Align all to common set of columns
final_columns = set(df_lb_std.columns) | set(df_sj_std.columns) | set(df_bloom_std.columns)
df_lb_std = df_lb_std.reindex(columns=final_columns)
df_sj_std = df_sj_std.reindex(columns=final_columns)
df_bloom_std = df_bloom_std.reindex(columns=final_columns)

### Separate Data to address intakedate is always the same in Bloomtington dataset ###
# Separate rows with duplicated IDs and without duplicated IDs
duplicated_bloom_mask = df_bloom_std.duplicated(subset="animal_id", keep=False)

# DataFrame with duplicated IDs
duplicated_bloom_df = df_bloom_std[duplicated_bloom_mask].copy()

# DataFrame without duplicated IDs
unique_bloom_df = df_bloom_std[~duplicated_bloom_mask].copy()

# Convert date columns to datetime
duplicated_bloom_df["intake_date"] = pd.to_datetime(duplicated_bloom_df["intake_date"], errors="coerce")
duplicated_bloom_df["return_date"] = pd.to_datetime(duplicated_bloom_df["return_date"], errors="coerce")
duplicated_bloom_df["outcome_date"] = pd.to_datetime(duplicated_bloom_df["outcome_date"], errors="coerce")

# Sort to ensure consistent ordering
duplicated_bloom_df.sort_values(by=["animal_id", "intake_date", "outcome_date"], inplace=True)

# Create a new column for adjusted intakedate
adjusted_intakedate = []

# Track previous returndate for each id
prev_returndate = {}

# Adjust intakedate for repeated IDs
for _, row in duplicated_bloom_df.iterrows():
    animal_id = row["animal_id"]
    if animal_id in prev_returndate:
        new_intakedate = prev_returndate[animal_id]
    else:
        new_intakedate = row["intake_date"]

    adjusted_intakedate.append(new_intakedate)

    # Update returndate if present
    if pd.notnull(row["return_date"]):
        prev_returndate[animal_id] = row["return_date"]

# Assign the adjusted intakedate
duplicated_bloom_df["intake_date"] = adjusted_intakedate

### Combine dataset ###
# Convert intakedate to datetime to match duplicated_df
unique_bloom_df["intake_date"] = pd.to_datetime(unique_bloom_df["intake_date"], errors="coerce")

# Add adjusted_intakedate column with original intakedate for unique entries
#unique_bloom_df["intake_date"] = unique_bloom_df["intake_date"]

# Combine both DataFrames
df_bloom_std = pd.concat([unique_bloom_df, duplicated_bloom_df], ignore_index=True)

# Sort for better readability
df_bloom_std.sort_values(by=["animal_id", "intake_date"], inplace=True)
### Finish Bloomtington "intake_date" correction ###

### Fill Bloomington 'outcome_type' blanks based on 'diedoffshelter', 'puttosleep', and 'isdoa' ###
def fill_bloomington_outcome(row):
    if pd.notna(row['outcome_type']):
        return row['outcome_type']
    if row.get('diedoffshelter') is True or row.get('isdoa') is True:
        return 'DIED'
    if row.get('puttosleep') is True:
        return 'EUTH'
    return 'UNKNOWN'

df_bloom_std['outcome_type'] = df_bloom_std.apply(fill_bloomington_outcome, axis=1)
df_bloom_std['outcome_type'] = df_bloom_std['outcome_type'].astype(str).str.upper()

### Finished filling blanks for Bloomington 'outcome_type'


# Combine all datasets
df_cat_dog = pd.concat([df_lb_std, df_sj_std, df_bloom_std], ignore_index=True)

# Show result
df_cat_dog.shape, list(df_cat_dog.columns), df_cat_dog['animal_type'].value_counts()

/tmp/ipython-input-16-1821603612.py:110: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  duplicated_bloom_df["intake_date"] = pd.to_datetime(duplicated_bloom_df["intake_date"], errors="coerce")
/tmp/ipython-input-16-1821603612.py:111: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  duplicated_bloom_df["return_date"] = pd.to_datetime(duplicated_bloom_df["return_date"], errors="coerce")
/tmp/ipython-input-16-1821603612.py:142: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  unique_bloom_df["intake_date"] = pd.to_datetime(unique_bloom_df["intake_date"], errors="coerce")


((137321, 41),
 ['sheltercode',
  'outcome_is_dead',
  'primary_breed',
  'identichipnumber',
  'LastUpdate',
  'outcome_year',
  'outcome_date',
  'Crossing',
  'shelter',
  'outcome_type',
  'location',
  'diedoffshelter',
  'geopoint',
  'puttosleep',
  'OutcomeCondition',
  'deceased_date',
  'Jurisdiction',
  'animal_name',
  'intake_condition',
  'was_outcome_alive',
  'primary_color',
  'Age',
  'intake_subtype',
  'istrial',
  'intake_type',
  'secondary_color',
  'sex',
  'dob',
  'longitude',
  'outcome_subtype',
  'returnedreason',
  'intake_is_dead',
  'latitude',
  'istransfer',
  'return_date',
  'animal_id',
  'animal_type',
  'intake_date',
  'deceasedreason',
  'isdoa',
  'intake_reason'],
 animal_type
 cat    95145
 dog    42176
 Name: count, dtype: int64)

In [ ]:
# Define a function to extract primary, secondary breed and mix flag
def extract_breed_parts(breed):
    if pd.isna(breed):
        return pd.Series(["unknown", None, "no"])

    parts = breed.split("/")
    primary = parts[0].strip()

    if len(parts) > 1:
        secondary = parts[1].strip()
        is_mix = "yes"
    else:
        secondary = None
        is_mix = "no"

    return pd.Series([primary, secondary, is_mix])

# Apply to df_cat_dog using 'primary_breed'
df_cat_dog[["primary_breed_clean", "secondary_breed", "is_mix"]] = df_cat_dog["primary_breed"].apply(extract_breed_parts)


In [ ]:
print(df_cat_dog[df_cat_dog["animal_id"] == 64598].head(20))

       sheltercode outcome_is_dead primary_breed identichipnumber LastUpdate  \
135882    D2307812             NaN     Akita/Mix  981020031845989        NaN   
135883    D2307812             NaN     Akita/Mix  981020031845989        NaN   
135884    D2307812             NaN     Akita/Mix  981020031845989        NaN   
135885    D2307812             NaN     Akita/Mix  981020031845989        NaN   
135886    D2307812             NaN     Akita/Mix  981020031845989        NaN   
135887    D2307812             NaN     Akita/Mix  981020031845989        NaN   
135888    D2307812             NaN     Akita/Mix  981020031845989        NaN   
135889    D2307812             NaN     Akita/Mix  981020031845989        NaN   

        outcome_year outcome_date Crossing      shelter outcome_type  ...  \
135882        2019.0   2019-08-31      NaN  Bloomington     ADOPTION  ...   
135883        2021.0   2021-05-06      NaN  Bloomington     ADOPTION  ...   
135884        2023.0   2023-09-15      NaN  Bloo

In [ ]:
print(df_cat_dog.columns)


Index(['sheltercode', 'outcome_is_dead', 'primary_breed', 'identichipnumber',
       'LastUpdate', 'outcome_year', 'outcome_date', 'Crossing', 'shelter',
       'outcome_type', 'location', 'diedoffshelter', 'geopoint', 'puttosleep',
       'OutcomeCondition', 'deceased_date', 'Jurisdiction', 'animal_name',
       'intake_condition', 'was_outcome_alive', 'primary_color', 'Age',
       'intake_subtype', 'istrial', 'intake_type', 'secondary_color', 'sex',
       'dob', 'longitude', 'outcome_subtype', 'returnedreason',
       'intake_is_dead', 'latitude', 'istransfer', 'return_date', 'animal_id',
       'animal_type', 'intake_date', 'deceasedreason', 'isdoa',
       'intake_reason', 'primary_breed_clean', 'secondary_breed', 'is_mix'],
      dtype='object')


## Continue Preprocessing and Cleaning

### a. Process NAs

#### i. Covert None/Null/blank to NaN for columns: intake_subtype, intake_reason, secondary_color, age

In [ ]:
# List of columns to clean
cols_to_clean = ['intake_subtype', 'intake_reason', 'secondary_color', 'Age']

# Apply replacement
for col in cols_to_clean:
    df_cat_dog[col] = (
        df_cat_dog[col]
        .replace(r'^\s*$', np.nan, regex=True)  # blank or whitespace -> NaN
        .replace(r'(?i)^none$', np.nan, regex=True)  # 'None' (any case) -> NaN
    )


In [ ]:
nan_count_intake_reason = df_cat_dog['intake_reason'].isna().sum()
print(f"Total NaN in intake_reason: {nan_count_intake_reason}")
nan_count_Age = df_cat_dog['Age'].isna().sum()
print(f"Total NaN in Age: {nan_count_Age}")

Total NaN in intake_reason: 117447
Total NaN in Age: 17648


#### ii. Remove "outcome_type" is blank

In [ ]:
# Drop rows where outcome_type is NaN
df_cat_dog_final = df_cat_dog[df_cat_dog['outcome_type'].notna()]

# Show the new shape
print(f"New shape after removing rows with outcome_type NaN: {df_cat_dog_final.shape}")

New shape after removing rows with outcome_type NaN: (136185, 44)


### b. Remove Duplicates

#### i. Drop Exact Duplicates

In [ ]:
# Check for exact duplicates across all columns
duplicate_rows = df_cat_dog_final[df_cat_dog_final.duplicated(keep=False)]

# Show how many duplicates found
print(f"Number of exact duplicate rows: {duplicate_rows.shape[0]}")

# Drop exact duplicates (keep the first occurrence)
df_cat_dog_no_duplicates = df_cat_dog_final.drop_duplicates(keep='first')

# Show new shape after dropping duplicates
print(f"New shape after removing duplicates: {df_cat_dog_no_duplicates.shape}")


Number of exact duplicate rows: 214
New shape after removing duplicates: (136035, 44)


In [ ]:
print(duplicate_rows.head(20))

      sheltercode outcome_is_dead primary_breed identichipnumber LastUpdate  \
18711         NaN             NaN    POODLE MIN              NaN  3/14/2019   
18712         NaN             NaN    POODLE MIN              NaN  3/14/2019   
24100         NaN             NaN   DOMESTIC SH              NaN  9/21/2018   
24101         NaN             NaN   DOMESTIC SH              NaN  9/21/2018   
26126         NaN             NaN   DOMESTIC SH              NaN  5/15/2019   
26127         NaN             NaN   DOMESTIC SH              NaN  5/15/2019   
35964         NaN             NaN   DOMESTIC SH              NaN   3/2/2020   
35965         NaN             NaN   DOMESTIC SH              NaN   3/2/2020   
39968         NaN             NaN   DOMESTIC SH              NaN   9/1/2019   
39969         NaN             NaN   DOMESTIC SH              NaN   9/1/2019   
61230         NaN             NaN   DOMESTIC SH              NaN   5/8/2021   
61231         NaN             NaN   DOMESTIC SH     

In [ ]:
print(df_cat_dog_final[df_cat_dog_final['animal_id'] == 'A0998965'])

      sheltercode outcome_is_dead primary_breed identichipnumber LastUpdate  \
18711         NaN             NaN    POODLE MIN              NaN  3/14/2019   
18712         NaN             NaN    POODLE MIN              NaN  3/14/2019   
18713         NaN             NaN    POODLE MIN              NaN   4/6/2019   

       outcome_year outcome_date                     Crossing   shelter  \
18711        2019.0   2019-03-14                  BARBERRY LN  San Jose   
18712        2019.0   2019-03-14                  BARBERRY LN  San Jose   
18713        2019.0   2019-04-06  BARBERRY LN. / STALLION WAY  San Jose   

      outcome_type  ... return_date animal_id animal_type intake_date  \
18711          RTO  ...         NaN  A0998965         dog   3/14/2019   
18712          RTO  ...         NaN  A0998965         dog   3/14/2019   
18713          RTO  ...         NaN  A0998965         dog    4/5/2019   

      deceasedreason isdoa intake_reason primary_breed_clean secondary_breed  \
18711    

In [ ]:
print(df_cat_dog_no_duplicates[df_cat_dog_no_duplicates['animal_id'] == 'A0998965'])

      sheltercode outcome_is_dead primary_breed identichipnumber LastUpdate  \
18711         NaN             NaN    POODLE MIN              NaN  3/14/2019   
18713         NaN             NaN    POODLE MIN              NaN   4/6/2019   

       outcome_year outcome_date                     Crossing   shelter  \
18711        2019.0   2019-03-14                  BARBERRY LN  San Jose   
18713        2019.0   2019-04-06  BARBERRY LN. / STALLION WAY  San Jose   

      outcome_type  ... return_date animal_id animal_type intake_date  \
18711          RTO  ...         NaN  A0998965         dog   3/14/2019   
18713          RTO  ...         NaN  A0998965         dog    4/5/2019   

      deceasedreason isdoa intake_reason primary_breed_clean secondary_breed  \
18711            NaN   NaN           NaN          POODLE MIN            None   
18713            NaN   NaN      IP ADOPT          POODLE MIN            None   

       is_mix  
18711      no  
18713      no  

[2 rows x 44 columns]


#### ii. Drop non exact duplictes based on ID+INTAKETIME+OUTCOMETYPE+shelter

In [ ]:
# Ensure intake_date is string for grouping (in case it's datetime)
df_cat_dog_no_duplicates['intake_date_str'] = df_cat_dog_no_duplicates['intake_date'].astype(str)

# Define the subset of columns to check duplicates on
subset_cols = ['animal_id', 'intake_date_str', 'outcome_type', 'shelter']

# Find duplicates based on these fields
duplicate_subset = df_cat_dog_no_duplicates[df_cat_dog_no_duplicates.duplicated(subset=subset_cols, keep=False)]

print(f"Number of non-exact duplicates based on key fields: {duplicate_subset.shape[0]}")

# Drop duplicates, keep first occurrence
df_cat_dog_clean = df_cat_dog_no_duplicates.drop_duplicates(subset=subset_cols, keep='first')

print(f"New shape after removing non-exact duplicates: {df_cat_dog_clean.shape}")


Number of non-exact duplicates based on key fields: 482


/tmp/ipython-input-27-1303364740.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat_dog_no_duplicates['intake_date_str'] = df_cat_dog_no_duplicates['intake_date'].astype(str)


New shape after removing non-exact duplicates: (135787, 45)


In [ ]:
print(df_cat_dog_clean[df_cat_dog_clean['animal_id'] == 'A670669'])
print(df_cat_dog_no_duplicates[df_cat_dog_no_duplicates['animal_id'] == 'A670669'])

     sheltercode outcome_is_dead primary_breed identichipnumber LastUpdate  \
7096         NaN           False           NaN              NaN        NaN   

      outcome_year outcome_date                                     Crossing  \
7096        2022.0   2022-05-22  3300 BLK SANTA FE AVE, LONG BEACH, CA 90810   

         shelter outcome_type  ... animal_id animal_type intake_date  \
7096  Long Beach     ADOPTION  ...   A670669         cat   2/14/2022   

     deceasedreason isdoa intake_reason primary_breed_clean secondary_breed  \
7096            NaN   NaN    OWNER PROB             unknown            None   

     is_mix  intake_date_str  
7096     no        2/14/2022  

[1 rows x 45 columns]
     sheltercode outcome_is_dead primary_breed identichipnumber LastUpdate  \
7096         NaN           False           NaN              NaN        NaN   
7109         NaN           False           NaN              NaN        NaN   

      outcome_year outcome_date                           

In [ ]:
print(duplicate_subset.head(20))

      sheltercode outcome_is_dead   primary_breed identichipnumber  \
4476          NaN           False             NaN              NaN   
7096          NaN           False             NaN              NaN   
7109          NaN           False             NaN              NaN   
7197          NaN           False             NaN              NaN   
8086          NaN           False             NaN              NaN   
8088          NaN           False             NaN              NaN   
10084         NaN           False             NaN              NaN   
11250         NaN           False             NaN              NaN   
17673         NaN             NaN   GERM SHEPHERD              NaN   
17674         NaN             NaN   GERM SHEPHERD              NaN   
23645         NaN             NaN     DOMESTIC SH              NaN   
23646         NaN             NaN     DOMESTIC SH              NaN   
26188         NaN             NaN     DOMESTIC SH              NaN   
26189         NaN   

# Feature Engineered Columns:
  a. Has name \
  b. Age \
  c. Lenght of stay \
  d. is_return \
  e. Num_returned \
  f. Tag Primary Breed for mixed breed \

## Feature "Has Name" from column "animal_name"

#### Calculated Field for 'has_name'

In [ ]:
# Create a binary field: 1 = named, 0 = unnamed (empty or 'nan' or similar treated as unnamed)
def has_name(value):
    value = str(value).strip()
    if value.lower() in ['nan', '', 'none', 'unknown']:
        return 0
    else:
        return 1

df_cat_dog_clean['has_name'] = df_cat_dog_clean['animal_name'].apply(has_name)

# Show distribution of named vs unnamed
name_counts = df_cat_dog_clean['has_name'].value_counts().rename(index={1: 'named', 0: 'unnamed'})
name_counts


/tmp/ipython-input-30-4189929964.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat_dog_clean['has_name'] = df_cat_dog_clean['animal_name'].apply(has_name)


,count
has_name,
named,74607
unnamed,61180


## Feature "Stay Length" from column "intakedate" and "outcomedate"

In [ ]:
# First ensure dates are converted to datetime if not already
df_cat_dog_clean['intake_date'] = pd.to_datetime(df_cat_dog_clean['intake_date'], errors='coerce')
df_cat_dog_clean['outcome_date'] = pd.to_datetime(df_cat_dog_clean['outcome_date'], errors='coerce')

# Calculate stay length in days
df_cat_dog_clean['stay_length_days'] = (df_cat_dog_clean['outcome_date'] - df_cat_dog_clean['intake_date']).dt.total_seconds() / (60*60*24)

# Replace negative stay lengths with 0
df_cat_dog_clean['stay_length_days'] = df_cat_dog_clean['stay_length_days'].clip(lower=0)

# Show basic stats for stay length
stay_length_summary = df_cat_dog_clean['stay_length_days'].describe()
stay_length_summary


/tmp/ipython-input-31-2106714349.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat_dog_clean['intake_date'] = pd.to_datetime(df_cat_dog_clean['intake_date'], errors='coerce')
/tmp/ipython-input-31-2106714349.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat_dog_clean['outcome_date'] = pd.to_datetime(df_cat_dog_clean['outcome_date'], errors='coerce')
/tmp/ipython-input-31-2106714349.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

,stay_length_days
count,135753.000000
mean,12.251958
std,39.861157
min,0.000000
25%,0.456748
50%,4.000000
75%,11.000000
max,3653.000000


## Feature "Is_returned" and 'Num_returned'

In [ ]:
# Step 1: Ensure the DataFrame is sorted by animal_id and intake_date
df_cat_dog_clean = df_cat_dog_clean.sort_values(by=['animal_id', 'intake_date'])

# Step 2: Use groupby().cumcount() to efficiently count returns per animal_id
df_cat_dog_clean['Num_returned'] = df_cat_dog_clean.groupby('animal_id').cumcount()

# Step 3: Create Is_returned column based on Num_returned
df_cat_dog_clean['Is_returned'] = df_cat_dog_clean['Num_returned'].apply(lambda x: 'no' if x == 0 else 'yes')


In [ ]:
#df_cat_dog_clean['Is_returned'] = df_cat_dog_clean['Num_returned'].apply(lambda x: 'Yes' if x > 0 else 'No')

In [ ]:
print(df_cat_dog_clean.shape)
print(df_cat_dog_clean.columns)

(135787, 49)
Index(['sheltercode', 'outcome_is_dead', 'primary_breed', 'identichipnumber',
       'LastUpdate', 'outcome_year', 'outcome_date', 'Crossing', 'shelter',
       'outcome_type', 'location', 'diedoffshelter', 'geopoint', 'puttosleep',
       'OutcomeCondition', 'deceased_date', 'Jurisdiction', 'animal_name',
       'intake_condition', 'was_outcome_alive', 'primary_color', 'Age',
       'intake_subtype', 'istrial', 'intake_type', 'secondary_color', 'sex',
       'dob', 'longitude', 'outcome_subtype', 'returnedreason',
       'intake_is_dead', 'latitude', 'istransfer', 'return_date', 'animal_id',
       'animal_type', 'intake_date', 'deceasedreason', 'isdoa',
       'intake_reason', 'primary_breed_clean', 'secondary_breed', 'is_mix',
       'intake_date_str', 'has_name', 'stay_length_days', 'Num_returned',
       'Is_returned'],
      dtype='object')


In [ ]:
df_cat_dog_clean[df_cat_dog_clean['animal_id'] == 'A1083706'].sort_values(by='intake_date')

,sheltercode,outcome_is_dead,primary_breed,identichipnumber,LastUpdate,outcome_year,outcome_date,Crossing,shelter,outcome_type,...,isdoa,intake_reason,primary_breed_clean,secondary_breed,is_mix,intake_date_str,has_name,stay_length_days,Num_returned,Is_returned
19417,NaN,NaN,SIBERIAN HUSKY,NaN,9/26/2018,2018.0,2018-09-26,"100 BLOCK S KING RD, SAN JOSE CA 95116",San Jose,RTO,...,NaN,IP ADOPT,SIBERIAN HUSKY,None,no,9/24/2018,1,2.0,0,no
19418,NaN,NaN,SIBERIAN HUSKY,NaN,12/2/2019,2019.0,2019-05-07,ALUM ROCK/ MCCREERY AVE,San Jose,RTO,...,NaN,NaN,SIBERIAN HUSKY,None,no,5/7/2019,1,0.0,1,yes
36184,NaN,NaN,SIBERIAN HUSKY,NaN,9/18/2019,2019.0,2019-09-18,KENTUCKY PL / ALUM ROCK,San Jose,RTO,...,NaN,NaN,SIBERIAN HUSKY,None,no,9/17/2019,1,1.0,2,yes
36185,NaN,NaN,SIBERIAN HUSKY,NaN,9/27/2019,2019.0,2019-09-27,KING RD AND MCKEE RD (PLATA ARRYO PARK),San Jose,RTO,...,NaN,NaN,SIBERIAN HUSKY,None,no,9/24/2019,1,3.0,3,yes
36187,NaN,NaN,SIBERIAN HUSKY,NaN,11/5/2019,2019.0,2019-11-05,WHITE RD/ ALUM ROCK RD,San Jose,RTO,...,NaN,NaN,SIBERIAN HUSKY,None,no,10/31/2019,1,5.0,4,yes
36189,NaN,NaN,SIBERIAN HUSKY,NaN,12/2/2019,2019.0,2019-11-15,1900 BLOCK ALUM ROCK AVE,San Jose,RTO,...,NaN,NaN,SIBERIAN HUSKY,None,no,11/15/2019,1,0.0,5,yes
36188,NaN,NaN,SIBERIAN HUSKY,NaN,12/2/2019,2019.0,2019-11-18,CASTLEBRIDGE DR,San Jose,RTO,...,NaN,NaN,SIBERIAN HUSKY,None,no,11/18/2019,1,0.0,6,yes
36186,NaN,NaN,SIBERIAN HUSKY,NaN,1/17/2020,2020.0,2020-01-17,STORY RD & KING RD,San Jose,ADOPTION,...,NaN,NaN,SIBERIAN HUSKY,None,no,12/3/2019,1,45.0,7,yes
46908,NaN,NaN,SIBERIAN HUSKY,NaN,6/2/2020,2020.0,2020-06-02,ALUM ROCK AVE X KING RD,San Jose,RTO,...,NaN,NaN,SIBERIAN HUSKY,None,no,6/2/2020,1,0.0,8,yes
50618,NaN,NaN,SIBERIAN HUSKY,NaN,7/1/2020,2020.0,2020-07-01,ALUM ROCK AVE & KING RD,San Jose,RTO,...,NaN,NaN,SIBERIAN HUSKY,None,no,7/1/2020,1,0.0,9,yes


## Feature 'age_months'

In [ ]:
# Define the conversion function
def parse_age_to_months(age_str):
    if pd.isna(age_str):
        return np.nan
    age_str = str(age_str).strip().lower()
    if age_str == 'no age':
        return np.nan

    match = re.match(r'(\d+)\s*(year|years|month|months|week|weeks|day|days)', age_str)
    if not match:
        return np.nan

    value = int(match.group(1))
    unit = match.group(2)

    if 'year' in unit:
        return value * 12
    elif 'month' in unit:
        return value
    elif 'week' in unit:
        return round(value / 4.345, 2)  # approx weeks to months
    elif 'day' in unit:
        return round(value / 30, 2)  # approx days to months
    else:
        return np.nan

# Apply to create new column
df_cat_dog_clean['age_months'] = df_cat_dog_clean['Age'].apply(parse_age_to_months)

In [ ]:
print(df_cat_dog_clean.shape)
print(df_cat_dog_clean.columns)

(135787, 50)
Index(['sheltercode', 'outcome_is_dead', 'primary_breed', 'identichipnumber',
       'LastUpdate', 'outcome_year', 'outcome_date', 'Crossing', 'shelter',
       'outcome_type', 'location', 'diedoffshelter', 'geopoint', 'puttosleep',
       'OutcomeCondition', 'deceased_date', 'Jurisdiction', 'animal_name',
       'intake_condition', 'was_outcome_alive', 'primary_color', 'Age',
       'intake_subtype', 'istrial', 'intake_type', 'secondary_color', 'sex',
       'dob', 'longitude', 'outcome_subtype', 'returnedreason',
       'intake_is_dead', 'latitude', 'istransfer', 'return_date', 'animal_id',
       'animal_type', 'intake_date', 'deceasedreason', 'isdoa',
       'intake_reason', 'primary_breed_clean', 'secondary_breed', 'is_mix',
       'intake_date_str', 'has_name', 'stay_length_days', 'Num_returned',
       'Is_returned', 'age_months'],
      dtype='object')


## From data sources that don’t have that column, i.e. breed List as “Unknown”


In [ ]:
# Replace NaN in primary_breed with "Unknown"
df_cat_dog_clean['primary_breed_clean'] = df_cat_dog_clean['primary_breed_clean'].fillna("Unknown")
df_cat_dog_cleaned = df_cat_dog_clean.copy()


In [ ]:
# Check how many "Unknown" values now exist
unknown_count = (df_cat_dog_cleaned['primary_breed_clean'] == "Unknown").sum()
print(f'Number of records with primary_breed = "Unknown": {unknown_count}')


Number of records with primary_breed = "Unknown": 0


# Data Harmonization

## Column Harmonizer Class Definitions

In [ ]:
class ColumnHarmonizer:
    """
    A class to standardize, harmonize, and evaluate a specific column
    in a DataFrame across different shelters.
    """
    def __init__(self, df: pd.DataFrame, column_name: str, shelter_column: str = 'shelter'):
        """
        Initializes the Harmonizer with the DataFrame and column to process.

        Args:
            df (pd.DataFrame): The input DataFrame containing shelter data.
            column_name (str): The name of the column to harmonize.
            shelter_column (str, optional): The column identifying the shelter. Defaults to 'shelter'.
        """
        if column_name not in df.columns or shelter_column not in df.columns:
            raise ValueError("Specified columns do not exist in the DataFrame.")

        self.df = df.copy()
        self.column_name = column_name
        self.shelter_column = shelter_column
        self.shelters = self.df[self.shelter_column].unique()

        # Store original and create a harmonized series placeholder
        self.df['original_values'] = self.df[column_name].astype(str)
        self.df['harmonized_values'] = self._normalize_series(self.df['original_values'])

        # Initialize component extraction settings for color harmonization
        self._init_color_components()

    def _init_color_components(self):
        """Initialize abbreviations and patterns for color harmonization."""
        self.abbreviations = {
            # Color abbreviations
            'brn': 'brown', 'br': 'brown', 'bn': 'brown',
            'blk': 'black', 'bl': 'blue',
            'org': 'orange', 'orn': 'orange', 'gry': 'gray', 'gr': 'gray',
            'crm': 'cream', 'cr': 'cream', 'slvr': 'silver', 'slv': 'silver',
            'choc': 'chocolate', 'ch': 'chocolate', 'lc': 'lilac', 'li': 'lilac',
            'sl': 'seal', 'st': 'seal', 'rd': 'red', 'lv': 'liver',
            'tn': 'tan', 'ln': 'lean', 'wh': 'white', 'yw': 'yellow', 'yl': 'yellow',
            # Pattern abbreviations
            'pt': 'point', 'dil': 'dilute', 'tab': 'tabby', 'brind': 'brindle',
            'smk': 'smoke', 'mut': 'mute',
            # Special combinations
            'b\\o': 'black orange', 'b/o': 'black orange', 'tri': 'tricolor'
        }

        self.patterns = ['calico', 'tortie', 'torbi', 'tabby', 'tiger', 'point',
                        'brindle', 'merle', 'lynx', 'smoke', 'shade', 'tick',
                        'snowshoe', 'dapple']

        self.colors = ['black', 'white', 'brown', 'gray', 'grey', 'orange', 'cream',
                      'blue', 'red', 'chocolate', 'seal', 'lilac', 'silver', 'gold',
                      'fawn', 'buff', 'tan', 'yellow', 'liver', 'apricot', 'wheat',
                      'peach', 'pink', 'beige', 'blonde', 'tricolor', 'flame']

        self.modifiers = ['dilute', 'smoke', 'shade', 'mute']

    def _normalize_series(self, series: pd.Series) -> pd.Series:
        """
        (Step 2) Applies a standard normalization process to a text series.
        """
        # Fill NaN, convert to uppercase, and strip whitespace
        normalized = series.fillna('unknown').str.upper().str.strip()
        # Replace common separators with a single space
        normalized = normalized.str.replace(r'[\-/_\\,.]', ' ', regex=True)
        # Replace multiple spaces with a single space
        normalized = normalized.str.replace(r'\s+', ' ', regex=True).str.strip()
        return normalized

    def _expand_abbreviations(self, text: str) -> str:
        """Expand known abbreviations in text."""
        expanded = text.lower()
        for abbr, full in sorted(self.abbreviations.items(), key=lambda x: len(x[0]), reverse=True):
            pattern = r'\b' + re.escape(abbr) + r'\b'
            expanded = re.sub(pattern, full, expanded)
        return expanded

    def _extract_components(self, text: str) -> Dict[str, List[str]]:
        """Extract semantic components from text (for colors)."""
        components = {'pattern': [], 'color': [], 'modifier': []}

        # First normalize and expand the text
        normalized = text.lower().strip()
        # Replace separators with spaces for processing
        normalized = re.sub(r'[-/\\,.]', ' ', normalized)
        normalized = re.sub(r'\s+', ' ', normalized).strip()

        # Expand abbreviations
        expanded = self._expand_abbreviations(normalized)

        # Split by spaces to get all words
        words = expanded.split()

        # Extract components
        used_words = set()

        # Extract patterns first
        for word in words:
            if word in self.patterns and word not in used_words:
                components['pattern'].append(word)
                used_words.add(word)

        # Extract modifiers
        for word in words:
            if word in self.modifiers and word not in used_words:
                components['modifier'].append(word)
                used_words.add(word)

        # Extract colors
        for word in words:
            if word in self.colors and word not in used_words:
                components['color'].append(word)
                used_words.add(word)

        # Special handling for compound colors that weren't expanded
        remaining = ' '.join(w for w in words if w not in used_words)
        if 'black orange' in remaining:
            components['color'].extend(['black', 'orange'])

        return components

    def _format_components(self, components: Dict[str, List[str]]) -> str:
        """Format components in standard order with alphabetical sorting."""
        parts = []

        # Sort and add each component type
        for comp_type in ['pattern', 'color', 'modifier']:
            if components.get(comp_type):
                sorted_items = sorted(components[comp_type])
                parts.append(' '.join(sorted_items).upper())

        return '-'.join(parts) if parts else 'UNKNOWN'

        """
        Applies a user-defined mapping to the normalized values.

        Args:
            mapping (Dict[str, str]): A dictionary where keys are the values to be
                                     replaced and values are the new harmonized values.
        """
        # Ensure mapping keys are normalized for matching
        normalized_mapping = {self._normalize_series(pd.Series([k])).iloc[0]: v for k, v in mapping.items()}
        self.df['harmonized_values'] = self.df['harmonized_values'].replace(normalized_mapping)

    def _get_shared_value_percentage(self, column_name: str) -> float:
        """
        Calculates the percentage of the most common value in a column.
        This is a proxy for how 'harmonized' the column is.
        """
        if column_name not in self.df.columns:
            return 0.0
        value_counts = self.df[column_name].value_counts()
        if value_counts.empty:
            return 0.0
        most_common = value_counts.iloc[0]
        total = len(self.df)
        return (most_common / total) * 100 if total > 0 else 0.0

    def evaluate_progress(self) -> Dict[str, float]:
        """
        Evaluates harmonization progress, comparing shared value percentages
        and the total number of unique values before and after.
        """
        # Calculate shared value progress
        before_progress = self._get_shared_value_percentage('original_values')
        after_progress = self._get_shared_value_percentage('harmonized_values')

        # Calculate unique value counts
        unique_before = self.df['original_values'].nunique()
        unique_after = self.df['harmonized_values'].nunique()

        # Calculate percentage of records with shared harmonized values
        # Get harmonized values that are shared across all shelters
        shared_values = []
        for shelter in self.shelters:
            shelter_values = set(self.df[self.df[self.shelter_column] == shelter]['harmonized_values'].unique())
            if not shared_values:
                shared_values = shelter_values
            else:
                shared_values = shared_values.intersection(shelter_values)

        # Count records that have these shared values
        records_with_shared_values = self.df['harmonized_values'].isin(shared_values).sum()
        total_records = len(self.df)
        pct_records_harmonized = (records_with_shared_values / total_records * 100) if total_records > 0 else 0

        return {
            "unique_values_before": unique_before,
            "unique_values_after": unique_after,
            "progress_before (%)": round(before_progress, 2),
            "progress_after (%)": round(after_progress, 2),
            "improvement (%)": round(after_progress - before_progress, 2),
            "records_harmonized (%)": round(pct_records_harmonized, 2),
            "records_harmonized_count": records_with_shared_values
        }

    def get_harmonization_report(self) -> pd.DataFrame:
        """
        Returns a DataFrame showing how original values were mapped.
        """
        report = self.df.groupby(['original_values', 'harmonized_values']).size().reset_index(name='count')
        return report.sort_values(['harmonized_values', 'count'], ascending=[False, False])

    def apply_mapping(self, mapping: Dict[str, str]):
        """
        Applies a user-defined mapping to the normalized values.

        Args:
            mapping (Dict[str, str]): A dictionary where keys are the values to be
                                      replaced and values are the new harmonized values.
        """
        # Ensure mapping keys are normalized for matching
        normalized_mapping = {
            self._normalize_series(pd.Series([k])).iloc[0]: v for k, v in mapping.items()
        }
        self.df['harmonized_values'] = self.df['harmonized_values'].replace(normalized_mapping)

    def apply_color_harmonization(self):
        """
        Applies semantic parsing (color, pattern, modifier) to harmonize color-related values.
        This should be called if working with color columns.
        """
        self.df['harmonized_values'] = self.df['harmonized_values'].apply(
            lambda x: self._format_components(self._extract_components(x))
        )


In [ ]:
# Step 1: Analyze what's new
def analyze_new_shelter(df, shelter_name, column_name):
    shelter_data = df[df['shelter'] == shelter_name][column_name].value_counts()
    print(f"\n{shelter_name} - {column_name} distribution:")
    return shelter_data

# Step 2: Update mappings if needed
def update_mappings_for_shelter(df, shelter_name, column_name, existing_mappings):
    harmonizer = ColumnHarmonizer(df, column_name)
    report = harmonizer.get_harmonization_report()

    # Filter for new shelter
    new_shelter_values = df[df['shelter'] == shelter_name][column_name].unique()

    # Check which values need mapping
    needs_mapping = []
    for val in new_shelter_values:
        if pd.notna(val) and val.upper() not in existing_mappings:
            needs_mapping.append(val)

    return print(f"Values in new shelter that still needs mapping: \n{needs_mapping}")

# Step 3: Re-harmonize all data
def harmonize_all_shelters(df, column_name, mappings):
    harmonizer = ColumnHarmonizer(df, column_name)
    harmonizer.apply_mapping(mappings)

    # Show progress
    progress = harmonizer.evaluate_progress()
    print(f"\nHarmonization Progress for {column_name}:")
    print(f"Unique values: {progress['unique_values_before']} → {progress['unique_values_after']}")
    print(f"Shared across all shelters: {progress['progress_before (%)']}% → {progress['progress_after (%)']}%")

    return harmonizer

In [ ]:
def compare_shelter_counts(
    df: pd.DataFrame,
    column_to_compare: str,
    shelter_names: Optional[List[str]] = None,
    shelter_column: str = 'shelter',
    min_count: int = 0,
    show_percentages: bool = False
) -> pd.DataFrame:
    """
    Compares the value counts of a specified column across multiple shelters.

    Args:
        df (pd.DataFrame): The input DataFrame containing data from multiple shelters.
        column_to_compare (str): The name of the column whose value counts are to be compared.
        shelter_names (List[str], optional): List of shelter names to compare.
            If None, compares all shelters in the dataset.
        shelter_column (str, optional): The name of the column identifying the shelter.
            Defaults to 'shelter'.
        min_count (int, optional): Minimum count threshold to include in results.
            Defaults to 0 (show all).
        show_percentages (bool, optional): If True, adds percentage columns for each shelter.
            Defaults to False.

    Returns:
        pd.DataFrame: A DataFrame comparing the value counts across all specified shelters.
    """
    # If no specific shelters specified, use all unique shelters
    if shelter_names is None:
        shelter_names = sorted(df[shelter_column].unique())

    # Validate shelter names exist in data
    available_shelters = set(df[shelter_column].unique())
    invalid_shelters = set(shelter_names) - available_shelters
    if invalid_shelters:
        raise ValueError(f"Shelters not found in data: {invalid_shelters}")

    # Start with the first shelter as base
    comparison_df = None

    # Process each shelter
    for shelter_name in shelter_names:
        # Get value counts for this shelter
        shelter_counts = df[
            df[shelter_column] == shelter_name
        ][column_to_compare].value_counts().reset_index()

        shelter_counts.columns = [column_to_compare, f'{shelter_name} Count']

        # Add percentage column if requested
        if show_percentages:
            total = shelter_counts[f'{shelter_name} Count'].sum()
            shelter_counts[f'{shelter_name} %'] = (
                shelter_counts[f'{shelter_name} Count'] / total * 100
            ).round(2)

        # Merge with existing comparison
        if comparison_df is None:
            comparison_df = shelter_counts
        else:
            comparison_df = pd.merge(
                comparison_df,
                shelter_counts,
                on=column_to_compare,
                how='outer'
            )

    # Fill NaN values with 0 for counts
    count_columns = [col for col in comparison_df.columns if 'Count' in col]
    comparison_df[count_columns] = comparison_df[count_columns].fillna(0).astype(int)

    # Fill NaN values with 0.0 for percentages
    if show_percentages:
        pct_columns = [col for col in comparison_df.columns if '%' in col]
        comparison_df[pct_columns] = comparison_df[pct_columns].fillna(0.0)

    # Calculate total count across all shelters
    comparison_df['Total Count'] = comparison_df[count_columns].sum(axis=1)

    # Apply minimum count filter
    if min_count > 0:
        comparison_df = comparison_df[comparison_df['Total Count'] >= min_count]

    # Sort by total count descending
    comparison_df = comparison_df.sort_values('Total Count', ascending=False)

    # Reorder columns to put Total Count after the value column
    cols = comparison_df.columns.tolist()
    cols.remove('Total Count')
    cols.insert(1, 'Total Count')
    comparison_df = comparison_df[cols]

    return comparison_df.reset_index(drop=True)

In [ ]:
# Breed Harmonization function
def harmonize_dog_breeds(df_source: pd.DataFrame, df_akc: pd.DataFrame, threshold: int = 70) -> pd.DataFrame:
    """
    Performs end-to-end breed harmonization on a DataFrame in a single, self-contained function.

    Args:
        df_source: The original DataFrame (e.g., df_cat_dog_cleaned).
        df_akc: The DataFrame containing official AKC breed names.
        threshold: The minimum weighted fuzzy score (0-100) to consider a match valid.

    Returns:
        A new DataFrame with the 'primary_breed_harmonized' column.
    """
    # --- 1. Define a nested helper function for consistent cleaning ---
    def _clean_breed(breed_name: str) -> str:
        if not isinstance(breed_name, str) or pd.isna(breed_name) or breed_name == '0':
            return "UNKNOWN"
        cleaned = breed_name.upper()
        # Remove colors, special characters, and normalize whitespace
        cleaned = re.sub(r'\b(BLACK|WHITE|BROWN|YELLOW|CHOCOLATE|RED|BLUE|BRINDLE)\b', '', cleaned)
        cleaned = re.sub(r'[.,\'"()]', '', cleaned) # Also remove parentheses
        cleaned = cleaned.replace('-', ' ')
        cleaned = ' '.join(cleaned.split())
        return cleaned if cleaned else "UNKNOWN"

    # --- 2. Prepare Cleaned Source and Target Breed Lists ---
    print("Step 1: Cleaning source and AKC breed lists...")
    # Clean source breeds (dogs only)
    dog_breeds_source = df_source[df_source['animal_type'] == 'dog']['primary_breed_clean'].dropna().unique()
    cleaned_source_breeds = sorted({_clean_breed(b) for b in dog_breeds_source})

    # Clean AKC target breeds
    akc_breeds_target = df_akc['dog_breed_name'].dropna().unique()
    cleaned_akc_targets = sorted({_clean_breed(b) for b in akc_breeds_target})

    # --- 3. Perform Fuzzy Matching Logic ---
    print("Step 2: Calculating fuzzy matches for each unique breed...")
    weights = {'ratio': 0.20, 'token_sort_ratio': 0.40, 'token_set_ratio': 0.40}
    # This map will store the translation from a cleaned source breed to its harmonized AKC name
    cleaned_to_harmonized_map = {}

    for source_breed in tqdm(cleaned_source_breeds):
        if source_breed == 'UNKNOWN':
            cleaned_to_harmonized_map[source_breed] = 'UNKNOWN'
            continue

        best_match = source_breed
        best_score = 0

        # Find the best match from the cleaned AKC list
        for akc_target in cleaned_akc_targets:
            scores = {
                'ratio': fuzz.ratio(source_breed, akc_target),
                'token_sort_ratio': fuzz.token_sort_ratio(source_breed, akc_target),
                'token_set_ratio': fuzz.token_set_ratio(source_breed, akc_target)
            }
            weighted_score = sum(scores[metric] * weight for metric, weight in weights.items())
            if weighted_score > best_score:
                best_score = weighted_score
                best_match = akc_target

        # Apply the threshold to decide if the match is valid
        if best_score >= threshold:
            cleaned_to_harmonized_map[source_breed] = best_match
        else:
            cleaned_to_harmonized_map[source_breed] = source_breed # No change if below threshold

    # --- 4. Apply Harmonization to the Full DataFrame ---
    print("Step 3: Applying final harmonization to the DataFrame...")
    df_final = df_source.copy()

    # Create a temporary column of cleaned original breeds
    df_final['temp_cleaned'] = df_final['primary_breed_clean'].apply(_clean_breed)

    # Map the cleaned breeds to their final harmonized names
    df_final['primary_breed_harmonized'] = df_final['temp_cleaned'].map(cleaned_to_harmonized_map)

    # Ensure non-dogs are not affected by setting their harmonized name to their original name
    dog_mask = df_final['animal_type'] == 'dog'
    df_final.loc[~dog_mask, 'primary_breed_harmonized'] = df_final.loc[~dog_mask, 'primary_breed_clean']

    # Handle any remaining NaNs and drop the temporary column
    df_final['primary_breed_harmonized'].fillna(df_final['primary_breed_clean'], inplace=True)
    df_final.drop(columns=['temp_cleaned'], inplace=True)

    print("Harmonization complete.")
    return df_final

### Mapping dictionary to harmonize "primary_breed"`

In [ ]:
df_final_harmonized = harmonize_dog_breeds(df_cat_dog_cleaned, df_akc, threshold=70)

Step 1: Cleaning source and AKC breed lists...
Step 2: Calculating fuzzy matches for each unique breed...


  0%|          | 0/315 [00:00<?, ?it/s]

Step 3: Applying final harmonization to the DataFrame...
Harmonization complete.


/tmp/ipython-input-43-2148224218.py:83: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_final['primary_breed_harmonized'].fillna(df_final['primary_breed_clean'], inplace=True)


In [ ]:
df_final_harmonized[['shelter', 'primary_breed', 'primary_breed_clean', 'primary_breed_harmonized']].sample(20)

,shelter,primary_breed,primary_breed_clean,primary_breed_harmonized
28199,San Jose,DOMESTIC LH,DOMESTIC LH,DOMESTIC LH
107485,San Jose,PIT BULL,PIT BULL,PIT BULL
76206,San Jose,DOMESTIC SH,DOMESTIC SH,DOMESTIC SH
122850,San Jose,DOMESTIC SH,DOMESTIC SH,DOMESTIC SH
79808,San Jose,SIBERIAN HUSKY,SIBERIAN HUSKY,SIBERIAN HUSKY
39299,San Jose,DOMESTIC SH,DOMESTIC SH,DOMESTIC SH
45761,San Jose,DOMESTIC SH,DOMESTIC SH,DOMESTIC SH
33682,San Jose,ENG BULLDOG,ENG BULLDOG,BULLDOG
74274,San Jose,PIT BULL,PIT BULL,PIT BULL
112350,San Jose,DOMESTIC SH,DOMESTIC SH,DOMESTIC SH


#### Add harmonized column to dataframe (add this to production pipeline)

In [ ]:
df_cat_dog_cleaned = harmonize_dog_breeds(df_cat_dog_cleaned, df_akc, threshold=70)

Step 1: Cleaning source and AKC breed lists...
Step 2: Calculating fuzzy matches for each unique breed...


  0%|          | 0/315 [00:00<?, ?it/s]

Step 3: Applying final harmonization to the DataFrame...
Harmonization complete.


/tmp/ipython-input-43-2148224218.py:83: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_final['primary_breed_harmonized'].fillna(df_final['primary_breed_clean'], inplace=True)


Examine harmonization results

In [ ]:
df_cat_dog_cleaned[(df_cat_dog_cleaned['animal_type']=='dog')&(df_cat_dog_cleaned['primary_breed_harmonized']!='UNKNOWN')].sample(100)[['primary_breed', 'primary_breed_clean', 'primary_breed_harmonized']]

,primary_breed,primary_breed_clean,primary_breed_harmonized
95642,GERM SHEPHERD,GERM SHEPHERD,GERMAN SHEPHERD DOG
118529,GERM SHEPHERD,GERM SHEPHERD,GERMAN SHEPHERD DOG
72108,MASTIFF,MASTIFF,MASTIFF
133868,Beagle/Mix,Beagle,BEAGLE
46732,CHIHUAHUA SH,CHIHUAHUA SH,CHIHUAHUA
...,...,...,...
105504,MALTESE,MALTESE,MALTESE
44546,GERM SHEPHERD,GERM SHEPHERD,GERMAN SHEPHERD DOG
52006,POODLE MIN,POODLE MIN,POODLE MINIATURE
85555,SIBERIAN HUSKY,SIBERIAN HUSKY,SIBERIAN HUSKY


### Mapping dictionary to harmonize "intake_Condition" (Bloomington does not have a suitable field for this)

In [ ]:
intake_condition_mappings = {}

#### Run standardization

In [ ]:
# 1. Instantiate the class for the 'intake_condition' column
intake_condition_harmonizer = ColumnHarmonizer(df_cat_dog_cleaned, 'intake_condition')

# 2. Apply the defined mappings
intake_condition_harmonizer.apply_mapping(intake_condition_mappings)

# 3. Evaluate the progress
progress = intake_condition_harmonizer.evaluate_progress()
print("Harmonization Progress for 'intake_type':")
print(progress)

intake_condition_harmonizer.get_harmonization_report()

Harmonization Progress for 'intake_type':
{'unique_values_before': 36, 'unique_values_after': 36, 'progress_before (%)': np.float64(32.71), 'progress_after (%)': np.float64(32.71), 'improvement (%)': np.float64(0.0), 'records_harmonized (%)': np.float64(85.16), 'records_harmonized_count': np.int64(115635)}


,original_values,harmonized_values,count
34,WELFARE SEIZURES,WELFARE SEIZURES,53
33,UNHEALTHY,UNHEALTHY,338
32,UNDER AGE/WEIGHT,UNDER AGE WEIGHT,3993
31,SICK,SICK,1469
30,REHAB,REHAB,4
29,PREGNANT,PREGNANT,852
28,OTHER MED,OTHER MED,14851
27,NURSING,NURSING,9569
26,NORMAL,NORMAL,44418
35,nan,NAN,11871


#### Initial comparison

In [ ]:
compare_shelter_counts(df_cat_dog_cleaned, 'intake_condition')

,intake_condition,Total Count,Bloomington Count,Long Beach Count,San Jose Count
0,NORMAL,44418,0,8922,35496
1,HEALTHY,15367,0,0,15367
2,OTHER MED,14851,0,0,14851
3,DEAD,10899,0,0,10899
4,NURSING,9569,0,0,9569
5,MED R,6764,0,0,6764
6,UNDER AGE/WEIGHT,3993,0,3993,0
7,FERAL,3531,0,225,3306
8,MED SEV,2509,0,0,2509
9,INJURED,1890,0,0,1890


Groupings:
1. Behavioral
2. Medical
3. Injured
4. Normal
5. Under age/weight = nursing

Add Intake Condition Mapping

In [ ]:
# Intake Condition Harmonization Mappings

intake_condition_mappings = {
    # Normal/Healthy conditions
    'NORMAL': 'NORMAL',
    'HEALTHY': 'NORMAL',

    # Illness and Medical Conditions
    'SICK': 'MEDICAL',
    'UNHEALTHY': 'MEDICAL',
    'ILL MODERATETE': 'MEDICAL',  # Note: typo in original
    'MED R': 'MEDICAL',
    'MED M': 'MEDICAL',
    'MED SEV': 'MEDICAL',
    'MED EMERG': 'MEDICAL',
    'I/I REPORT': 'MEDICAL', # Assumed Injured/Ill Reported.
    'PREGNANT': 'MEDICAL',
    'OTHER MED': 'MEDICAL',
    'ILL MILD': 'MEDICAL',
    'ILL SEVERE': 'MEDICAL',

    # Behavioral conditions by severity
    'AGGRESSIVE': 'BEHAVIOR',
    'FEARFUL': 'BEHAVIOR',
    'FRACTIOUS': 'BEHAVIOR',
    'BEH U': 'BEHAVIOR',
    'BEH M': 'BEHAVIOR',
    'BEH R': 'BEHAVIOR',
    'BEHAVIOR MILD': 'BEHAVIOR',
    'BEHAVIOR MODERATE': 'BEHAVIOR',
    'BEHAVIOR SEVERE': 'BEHAVIOR',

    # Under age/weight = nursing
    'UNDER AGE/WEIGHT': 'UNDER AGE, WEIGHT, NURSING',
    'NURSING': 'UNDER AGE, WEIGHT, NURSING',

    # Special conditions
    'MANAGE': 'OTHER',
    'REHAB': 'OTHER',
    'WELFARE SEIZURES': 'OTHER',

    # Injured
    'INJURED SEVERE': 'INJURED',
    'INJURED MODERATE': 'INJURED',
    'INJURED MILD': 'INJURED'
}

#### Final Results

In [ ]:
# 1. Instantiate the class for the 'outcome_type' column
intake_condition_harmonizer = ColumnHarmonizer(df_cat_dog_cleaned, 'intake_condition')

# 2. Apply the defined mappings
intake_condition_harmonizer.apply_mapping(intake_condition_mappings)

# 3. Evaluate the progress
progress = intake_condition_harmonizer.evaluate_progress()
print("Harmonization Progress for 'intake_type':")
print(progress)

intake_condition_harmonizer.get_harmonization_report()

Harmonization Progress for 'intake_type':
{'unique_values_before': 36, 'unique_values_after': 10, 'progress_before (%)': np.float64(32.71), 'progress_after (%)': np.float64(44.03), 'improvement (%)': np.float64(11.32), 'records_harmonized (%)': np.float64(91.26), 'records_harmonized_count': np.int64(123916)}


,original_values,harmonized_values,count
27,NURSING,"UNDER AGE, WEIGHT, NURSING",9569
32,UNDER AGE/WEIGHT,"UNDER AGE, WEIGHT, NURSING",3993
34,WELFARE SEIZURES,OTHER,53
21,MANAGE,OTHER,4
30,REHAB,OTHER,4
26,NORMAL,NORMAL,44418
12,HEALTHY,NORMAL,15367
35,nan,NAN,11871
28,OTHER MED,MEDICAL,14851
24,MED R,MEDICAL,6764


#### Final comparison between shelters

In [ ]:
compare_shelter_counts(intake_condition_harmonizer.df, 'harmonized_values')

,harmonized_values,Total Count,Bloomington Count,Long Beach Count,San Jose Count
0,NORMAL,59785,0,8922,50863
1,MEDICAL,30929,0,2045,28884
2,"UNDER AGE, WEIGHT, NURSING",13562,0,3993,9569
3,NAN,11871,11871,0,0
4,DEAD,10899,0,0,10899
5,FERAL,3531,0,225,3306
6,INJURED,3382,0,1492,1890
7,BEHAVIOR,1633,0,698,935
8,AGED,134,0,107,27
9,OTHER,61,0,53,8


#### Add harmonization values to original df

In [ ]:
df_cat_dog_cleaned['intake_condition_harmonized'] = intake_condition_harmonizer.df['harmonized_values']

### Mapping dictionary to harmonize "intake_reason"

In [ ]:
# intake reason harmonization dictionary
intake_reason_mapping = {
    'owner surrender': 'owner surrender',
    'surrender': 'owner surrender',
    'stray': 'stray',
    'lost': 'stray',
    'found': 'stray',
    'abandon': 'stray',
    'confiscated': 'confiscated',
    'seized': 'confiscated',
    'euthanasia request': 'euthanasia request',
    'medical': 'medical',
    'injured': 'medical',
    'sick': 'medical',
    'behavior': 'behavioral',
    'aggressive': 'behavioral',
    'biting': 'behavioral',
    'court case': 'legal',
    'legal case': 'legal',
    'quarantine': 'quarantine',
    'transfer': 'transfer',
    'born in care': 'born in care',
    'other': 'other'
}


In [ ]:
# Harmonize intake_reason
intake_reason_harmonizer = ColumnHarmonizer(
    df=df_cat_dog_cleaned,
    column_name='intake_reason',
    shelter_column='shelter'
)

# Apply the mapping
intake_reason_harmonizer.apply_mapping(intake_reason_mapping)

# Evaluate progress
intake_reason_report = intake_reason_harmonizer.evaluate_progress()
print("Intake Reason Harmonization Report:")
print(intake_reason_report)

# Optional: Save result back to main DataFrame
df_cat_dog_cleaned['intake_reason_harmonized'] = intake_reason_harmonizer.df['harmonized_values']


Intake Reason Harmonization Report:
{'unique_values_before': 92, 'unique_values_after': 89, 'progress_before (%)': np.float64(85.48), 'progress_after (%)': np.float64(85.48), 'improvement (%)': np.float64(0.0), 'records_harmonized (%)': np.float64(90.03), 'records_harmonized_count': np.int64(122250)}


### Mapping dictionary to harmonize "primary_color"

In [ ]:
# Define primary color harmonization mapping
primary_color_harmonization = {
    'brn': 'brown',
    'br': 'brown',
    'blk': 'black',
    'bl': 'blue',
    'gry': 'gray',
    'gr': 'gray',
    'org': 'orange',
    'orn': 'orange',
    'crm': 'cream',
    'cr': 'cream',
    'slvr': 'silver',
    'slv': 'silver',
    'choc': 'chocolate',
    'ch': 'chocolate',
    'lc': 'lilac',
    'li': 'lilac',
    'rd': 'red',
    'lv': 'liver',
    'tn': 'tan',
    'wh': 'white',
    'yw': 'yellow',
    'yl': 'yellow',
    'tri': 'tricolor',
    'pt-choc': 'chocolate point',
    'pt-lilac': 'lilac point',
    'pt-lynx': 'lynx point',
    'merle-red': 'red merle',
    'brindle-bn': 'brown brindle',
    'brindle-bl': 'blue brindle',
    'tabby-brn': 'brown tabby',
    'tabby-buff': 'buff tabby',
    'tabby-gray': 'gray tabby',
    'tabby-org': 'orange tabby',
    'gray tabby': 'gray tabby',
    'calico-dil': 'calico',
    'calico-tri': 'calico',
    'calico tabby': 'calico',
    'tortie-b\\o': 'tortie',
    'tortie-dil': 'tortie'
}


In [ ]:
# Apply harmonization to primary_color
df_cat_dog_cleaned['primary_color_harmonized'] = (
    df_cat_dog_cleaned['primary_color']
    .fillna('unknown')
    .astype(str)
    .str.strip()
    .str.lower()
    .map(primary_color_harmonization)
    .fillna(df_cat_dog_cleaned['primary_color'].astype(str).str.strip().str.lower())
)


In [ ]:
df_cat_dog_cleaned['primary_color_harmonized']

,primary_color_harmonized
125333,tortoiseshell
125334,tan
125335,black and tan
125336,black and tan
125337,black and tan
...,...
3129,l-c pt
7004,brn tabby
4003,tan
14858,gray


### Mapping dictionary to harmonize `intake_type`

#### Analyze Bloomington
Recommend to only use Bloomington `intake_reason` to map to `intake_type`, as most `intake_condition` mappings that can be done (i.e. Rabies Monitoring -> Medical, Injured -> Injured, Biting, Behavior -> Behavior, are low sample size - below 200, and we cannot make assumptions about `condition` for categories with bigger size i.e. Stray)

In [ ]:
analyze_new_shelter(df_cat_dog_cleaned, 'Bloomington', 'intake_reason')


Bloomington - intake_reason distribution:


,count
intake_reason,
Stray,5811
Incompatible with owner lifestyle,1393
Litter relinquishment,1286
Unable to Afford,487
Moving,485
Unsuitable Accommodation,464
Abandoned,354
Police Assist,249
Transfer from Other Shelter,234


#### Compare shelter counts - intake condition

In [ ]:
#@title { vertical-output: true }

compare_shelter_counts(intake_condition_harmonizer.df, 'harmonized_values')

,harmonized_values,Total Count,Bloomington Count,Long Beach Count,San Jose Count
0,NORMAL,59785,0,8922,50863
1,MEDICAL,30929,0,2045,28884
2,"UNDER AGE, WEIGHT, NURSING",13562,0,3993,9569
3,NAN,11871,11871,0,0
4,DEAD,10899,0,0,10899
5,FERAL,3531,0,225,3306
6,INJURED,3382,0,1492,1890
7,BEHAVIOR,1633,0,698,935
8,AGED,134,0,107,27
9,OTHER,61,0,53,8


#### Compare shelter counts - intake type

In [ ]:
#@title { vertical-output: true }

# Reinstantiate ColumnHarmonizer for 'intake_type'
intake_type_harmonizer = ColumnHarmonizer(df_cat_dog_cleaned, 'intake_type')
intake_type_harmonizer.apply_mapping(intake_reason_mapping)
compare_shelter_counts(intake_type_harmonizer.df, 'harmonized_values')

,harmonized_values,Total Count,Bloomington Count,Long Beach Count,San Jose Count
0,stray,89468,0,14553,74915
1,NAN,11871,11871,0,0
2,DISPO REQ,10686,0,0,10686
3,FOSTER,6121,0,1,6120
4,OWNER SUR,4882,0,0,4882
5,S N CLINIC,3778,0,0,3778
6,CONFISCATE,2671,0,278,2393
7,owner surrender,1850,0,1850,0
8,RETURN,1712,0,226,1486
9,transfer,1368,0,0,1368


#### Bloomington intake_reason to intake_type mappings

In [ ]:
# Bloomington intake_reason to intake_type mappings

intake_type_mappings = {
    # Other shelters mapping
    'OWNER SUR': 'OWNER SURRENDER',
    'EUTH REQ': 'EUTHENASIA REQUIRED',
    'TRAP, NEUTER, RETURN': 'SPAY/NEUTER',
    'SPAY': 'SPAY/NEUTER',
    'NEUTER': 'SPAY/NEUTER',
    'S N CLINIC': 'SPAY/NEUTER', # Spay / Neuter Clinic,
    'ADOPTED ANIMAL RETURN': 'RETURN',

    # Owner Surrenders (various reasons)
    'Abandoned': 'OWNER SURRENDER', # Assumption made.
    'Litter relinquishment': 'OWNER SURRENDER',
    'Incompatible with owner lifestyle': 'OWNER SURRENDER',
    'Unsuitable Accommodation': 'OWNER SURRENDER',
    'Moving': 'OWNER SURRENDER',
    'Unable to Afford': 'OWNER SURRENDER',
    'Landlord issues': 'OWNER SURRENDER',
    'Owner Deceased': 'OWNER SURRENDER',
    'Incompatible with other pets': 'OWNER SURRENDER',
    'Allergies': 'OWNER SURRENDER',
    'Housing': 'OWNER SURRENDER',
    'Marriage/Relationship split': 'OWNER SURRENDER',
    'Financial Constraints': 'OWNER SURRENDER',
    'Behavior': 'OWNER SURRENDER',
    'Medical': 'OWNER SURRENDER',

    # Return/Adoption Returns
    'Return Adopt - Behavior': 'RETURN',
    'Return Adopt - Owner Emergency': 'RETURN',
    'Return Adopt - Incompatible with owner lifestyle': 'RETURN',
    'Return adopt - lifestyle issue': 'RETURN',
    'Return Adopt - Incompatible with other pets': 'RETURN',
    'Return Adopt - Housing': 'RETURN',
    'Return Adopt - Other': 'RETURN',
    'Return Adopt - Financial': 'RETURN',
    'Return Adopt - Medical': 'RETURN',

    # Confiscations and Seizures
    'Police Assist': 'CONFISCATE',
    'Impound or Seizure': 'CONFISCATE',
    'Abuse/ neglect': 'WELFARE SEIZED',
    'Biting': 'CONFISCATE',

    # Special Programs and Monitoring
    'Rabies Monitoring': 'QUARANTINE',
    'TNR - Trap/Neuter/Release': 'SPAY/NEUTER',

    # Other Categories
    'Transfer from Other Shelter': 'TRANSFER',
    'Owner requested Euthanasia': 'EUTHENASIA REQUIRED',
    'Injured Wildlife': 'WILDLIFE',
    'DOA': 'DISPO REQ'  # Assumption made: dead on arrival
}

#### Rename Bloomington intake_reason to intake_type

In [ ]:
analyze_new_shelter(df_cat_dog_cleaned, 'Bloomington', 'intake_reason')


Bloomington - intake_reason distribution:


,count
intake_reason,
Stray,5811
Incompatible with owner lifestyle,1393
Litter relinquishment,1286
Unable to Afford,487
Moving,485
Unsuitable Accommodation,464
Abandoned,354
Police Assist,249
Transfer from Other Shelter,234


In [ ]:
# prompt: Assign df_cat_dog intake_type values where shelter = 'Bloomington' to be equal to df_cat_dog intake_reason values where shelter = 'Bloomington'

df_cat_dog_cleaned.loc[df_cat_dog_cleaned['shelter'] == 'Bloomington', 'intake_type'] = df_cat_dog_cleaned.loc[df_cat_dog_cleaned['shelter'] == 'Bloomington', 'intake_reason']

#### Harmonize

In [ ]:
# 1. Instantiate the class for the 'outcome_type' column
intake_type_harmonizer = ColumnHarmonizer(df_cat_dog_cleaned, 'intake_type')

# 2. Apply the defined mappings
intake_type_harmonizer.apply_mapping(intake_type_mappings)

# 3. Evaluate the progress
progress = intake_type_harmonizer.evaluate_progress()
print("Harmonization Progress for 'intake_type':")
print(progress)

intake_type_harmonizer.get_harmonization_report()

Harmonization Progress for 'intake_type':
{'unique_values_before': 46, 'unique_values_after': 16, 'progress_before (%)': np.float64(65.89), 'progress_after (%)': np.float64(70.17), 'improvement (%)': np.float64(4.28), 'records_harmonized (%)': np.float64(85.77), 'records_harmonized_count': np.int64(116470)}


,original_values,harmonized_values,count
44,WILDLIFE,WILDLIFE,4
43,WELFARE SEIZED,WELFARE SEIZED,402
1,Abuse/ neglect,WELFARE SEIZED,31
38,TRANSFER,TRANSFER,1368
40,Transfer from Other Shelter,TRANSFER,234
35,STRAY,STRAY,89468
36,Stray,STRAY,5811
32,S/N CLINIC,SPAY/NEUTER,3778
39,"TRAP, NEUTER, RETURN",SPAY/NEUTER,43
37,TNR - Trap/Neuter/Release,SPAY/NEUTER,32


#### Final comparison between shelters

In [ ]:
compare_shelter_counts(intake_type_harmonizer.df, 'harmonized_values')

,harmonized_values,Total Count,Bloomington Count,Long Beach Count,San Jose Count
0,STRAY,95279,5811,14553,74915
1,OWNER SURRENDER,11779,5047,1850,4882
2,DISPO REQ,10687,1,0,10686
3,FOSTER,6121,0,1,6120
4,SPAY/NEUTER,3901,32,43,3826
5,CONFISCATE,3089,418,278,2393
6,RETURN,1742,29,227,1486
7,TRANSFER,1602,234,0,1368
8,EUTHENASIA REQUIRED,680,13,1,666
9,WELFARE SEIZED,433,31,402,0


#### Add harmonization values to original df

In [ ]:
df_cat_dog_cleaned['intake_type_harmonized'] = intake_type_harmonizer.df['harmonized_values']

### Mapping dictionary to harmonize "secondary_color"

In [ ]:
# First, list all unique secondary colors (lowercase, stripped) for review
sec_color_series = df_cat_dog_cleaned['secondary_color'].dropna().astype(str).str.strip().str.lower()
unique_sec_colors = sorted(sec_color_series.unique())

unique_sec_colors


['apricot',
 'beige',
 'bl brindle',
 'black',
 'blk smoke',
 'blk tabby',
 'blue',
 'blue tabby',
 'blue tick',
 'br brindle',
 'brindle',
 'brindle-bn',
 'brindle-ln',
 'brindle-tn',
 'brn merle',
 'brn tabby',
 'brn-gry',
 'brown',
 'brown tips',
 'buff',
 'calico',
 'calico-dil',
 'calico-tri',
 'chocolate',
 'cream',
 'fawn',
 'flame pt',
 'gold',
 'gray',
 'gray tabby',
 'grey',
 'light',
 'liver',
 'lynx pt',
 'marbled tabby',
 'merle-blue',
 'merle-red',
 'orange',
 'org tabby',
 'other',
 'pepper',
 'pt-choc',
 'pt-flame',
 'pt-lilac',
 'pt-lynx',
 'pt-seal',
 'red',
 'red tick',
 'sable',
 'shade-slv',
 'shade-smk',
 'silver',
 'tabby-brn',
 'tabby-buff',
 'tabby-gray',
 'tabby-org',
 'tan',
 'tick-black',
 'tick-yel',
 'torbi',
 'torbi-brn',
 'torbi-dil',
 'tortie',
 'tortie dil',
 'tortie-b\\o',
 'tortie-dil',
 'tricolor',
 'white',
 'wht',
 'yellow']

In [ ]:
# Define secondary color harmonization mapping
sec_color_harmonization = {
    'bl brindle': 'blue brindle',
    'blk tabby': 'black tabby',
    'br brindle': 'brown brindle',
    'brn tabby': 'brown tabby',
    'org tabby': 'orange tabby',
    'pt-choc': 'chocolate point',
    'pt-lilac': 'lilac point',
    'pt-lynx': 'lynx point',
    'merle-red': 'red merle',
    'brindle-bn': 'brown brindle',
    'brindle-bl': 'blue brindle',

    'tabby-brn': 'brown tabby',
    'tabby-buff': 'buff tabby',
    'tabby-gray': 'gray tabby',
    'tabby-org': 'orange tabby',
    'gray tabby': 'gray tabby',

    'calico-dil': 'calico',
    'calico-tri': 'calico',
    'calico tabby': 'calico',

    'tortie-b\\o': 'tortie',
    'tortie-dil': 'tortie'
}

In [ ]:
# Apply harmonization
df_cat_dog_cleaned['secondary_color_harmonized'] = (
    df_cat_dog_cleaned['secondary_color']
    .fillna('unknown')
    .astype(str)
    .str.strip()
    .str.lower()
    .map(sec_color_harmonization)
    .fillna(df_cat_dog_cleaned['secondary_color'].astype(str).str.strip().str.lower())
)

In [ ]:
# Show the top 5 harmonized secondary colors
final_sec_color_counts = df_cat_dog_cleaned['secondary_color_harmonized'].value_counts()
final_sec_color_counts.head()

,count
secondary_color_harmonized,
nan,82977
white,37294
black,5067
brown,3544
tan,2951


### Mapping dictionary to harmonize "outcome_type"

In [ ]:
# First inspect unique values in outcome_type
outcome_series = df_cat_dog_cleaned['outcome_type'].dropna().astype(str).str.strip().str.lower()
unique_outcomes = sorted(outcome_series.unique())

unique_outcomes


['adoption',
 'community cat',
 'died',
 'disposal',
 'duplicate',
 'escaped',
 'euth',
 'euthanasia',
 'foster',
 'foster to adopt',
 'found anim',
 'found exp',
 'homefirst',
 'lost exp',
 'missing',
 'neuter',
 'reclaimed',
 'released to wild',
 'req euth',
 'rescue',
 'retailer',
 'return to owner',
 'return to rescue',
 'return to wild habitat',
 'rtf',
 'rto',
 's/n unable',
 'shelter, neuter, return',
 'spay',
 'stolen',
 'transfer',
 'transport',
 'trap, neuter, release',
 'unknown']

In [ ]:
# Define outcome type harmonization mapping
outcome_haromnize = {
    'euth': 'euthanasia',
    'euthanasia': 'euthanasia',
    'req euth': 'euthanasia',

    'return to owner': 'return to owner',
    'reclaimed': 'return to owner',
    'rto': 'return to owner',

    'spay': 'neutered/spayed',
    'neuter': 'neutered/spayed',

    'found exp': 'lost/missing/escaped',
    'lost exp': 'lost/missing/escaped',
    'escaped': 'lost/missing/escaped',
    'stolen': 'lost/missing/escaped',
    'missing': 'lost/missing/escaped',

    'released to wild': 'released to wild',
    'community cat': 'released to wild',

    'foster': 'foster',
    'homefirst': 'foster',

    'transfer': 'transfer',
    'transport': 'transfer'

}

# Apply mapping
df_cat_dog_cleaned['outcome_type_harmonized'] = (
    df_cat_dog_cleaned['outcome_type']
    .fillna('unknown')
    .astype(str)
    .str.strip()
    .str.lower()
    .map(outcome_haromnize)
    .fillna(df_cat_dog_cleaned['outcome_type'].astype(str).str.strip().str.lower())
)

# Show cleaned outcome distribution
outcome_final_counts = df_cat_dog_cleaned['outcome_type_harmonized'].value_counts()
outcome_final_counts


,count
outcome_type_harmonized,
adoption,33682
rescue,24279
transfer,21764
return to owner,14132
disposal,10905
euthanasia,10128
foster,9299
rtf,5451
died,2176


In [ ]:
df_cat_dog_cleaned['outcome_type_harmonized'].nunique()

22

In [ ]:
# Define the grouping logic
outcome_group_map = {
    'adoption': 'adopted',
    'foster to adopt': 'adopted',
    'return to owner': 'adopted',
}

# Apply the grouping
df_cat_dog_cleaned['outcome_type_harmonized_grouped'] = (
    df_cat_dog_cleaned['outcome_type_harmonized']
    .map(outcome_group_map)
    .fillna('non-adopted')
)

In [ ]:
df_cat_dog_cleaned['outcome_type_harmonized_grouped'].value_counts()

,count
outcome_type_harmonized_grouped,
non-adopted,87812
adopted,47975


## Harmonized dataframe

In [ ]:
# Create a deep copy of the cleaned DataFrame
df_cat_dog_harmonized = df_cat_dog_cleaned.copy()
print(df_cat_dog_harmonized.columns)

Index(['sheltercode', 'outcome_is_dead', 'primary_breed', 'identichipnumber',
       'LastUpdate', 'outcome_year', 'outcome_date', 'Crossing', 'shelter',
       'outcome_type', 'location', 'diedoffshelter', 'geopoint', 'puttosleep',
       'OutcomeCondition', 'deceased_date', 'Jurisdiction', 'animal_name',
       'intake_condition', 'was_outcome_alive', 'primary_color', 'Age',
       'intake_subtype', 'istrial', 'intake_type', 'secondary_color', 'sex',
       'dob', 'longitude', 'outcome_subtype', 'returnedreason',
       'intake_is_dead', 'latitude', 'istransfer', 'return_date', 'animal_id',
       'animal_type', 'intake_date', 'deceasedreason', 'isdoa',
       'intake_reason', 'primary_breed_clean', 'secondary_breed', 'is_mix',
       'intake_date_str', 'has_name', 'stay_length_days', 'Num_returned',
       'Is_returned', 'age_months', 'primary_breed_harmonized',
       'intake_condition_harmonized', 'intake_reason_harmonized',
       'primary_color_harmonized', 'intake_type_harmoniz

### Introduce cat breed mapping and sex mapping to the dataframe

In [ ]:
def harmonize_feature_values(df):
    """
    Applies specific value harmonization rules to the dataframe.
    """
    # Create a copy to avoid modifying the original dataframe in place
    df_copy = df.copy()

    # 1. Harmonize cat breeds
    breed_map = {
        "Domestic Short Hair": "DOMESTIC SH",
        "Domestic Medium Hair": "DOMESTIC MH",
        "Domestic Long Hair": "DOMESTIC LH",
        "DSH": "DOMESTIC SH",
        "DMH": "DOMESTIC MH",
        "DLH": "DOMESTIC LH"
    }
    if 'primary_breed_harmonized' in df_copy.columns:
        df_copy['primary_breed_harmonized'] = df_copy['primary_breed_harmonized'].replace(breed_map)

    # 2. Harmonize 'sex' column
    sex_map = {
        "Female": "FEMALE",
        "Male": "MALE",
        "Unknown": "UNKNOWN",
        "SPAYED": "FEMALE",  # Group SPAYED under FEMALE
        "NEUTERED": "MALE",    # Group NEUTERED under MALE
        "Neutered": "MALE",
        "Spayed": "FEMALE"
    }
    if 'sex' in df_copy.columns:
        df_copy['sex'] = df_copy['sex'].replace(sex_map)

    return df_copy

# Apply the harmonization function to all three dataframes
#train_df_harmonized = harmonize_feature_values(train_df_harmonized)
#val_df_harmonized = harmonize_feature_values(val_df_harmonized)
#test_df_harmonized = harmonize_feature_values(test_df_harmonized)
df_cat_dog_harmonized = harmonize_feature_values(df_cat_dog_harmonized)

print("Harmonization complete.")

# --- Verification Step ---
# Let's check the value counts in the test set to confirm the changes
print("\\n--- Verifying 'sex' column in Bloomington data ---")
print(df_cat_dog_harmonized['sex'].value_counts())

print("\\n--- Verifying 'primary_breed_harmonized' for cats in Bloomington data ---")
# Filtering for cats to check the specific values
print(df_cat_dog_harmonized[df_cat_dog_harmonized['animal_type']=='cat']['primary_breed_harmonized'].value_counts().head())

Harmonization complete.
\n--- Verifying 'sex' column in Bloomington data ---
sex
MALE       60215
FEMALE     55974
UNKNOWN    19598
Name: count, dtype: int64
\n--- Verifying 'primary_breed_harmonized' for cats in Bloomington data ---
primary_breed_harmonized
DOMESTIC SH    71859
unknown        10782
DOMESTIC MH     7925
DOMESTIC LH     2767
SIAMESE          239
Name: count, dtype: int64


### Harmonize Animal DOB data

In [ ]:
# Define LB animal age_month based on DOB and outcome_date
# Convert dates to datetime
df_cat_dog_harmonized['dob'] = pd.to_datetime(df_cat_dog_harmonized['dob'], errors='coerce')
df_cat_dog_harmonized['outcome_date'] = pd.to_datetime(df_cat_dog_harmonized['outcome_date'], errors='coerce')

# Calculate age in months where shelter is Long Beach
df_cat_dog_harmonized.loc[df_cat_dog_harmonized['shelter'] == 'Long Beach', 'age_months'] = (
    (df_cat_dog_harmonized.loc[df_cat_dog_harmonized['shelter'] == 'Long Beach', 'outcome_date'] - df_cat_dog_harmonized.loc[df_cat_dog_harmonized['shelter'] == 'Long Beach', 'dob'])
    .dt.days // 30
).clip(lower=0)  # Clip to ensure no negative ages

# Preview the result
df_cat_dog_harmonized[['shelter', 'dob', 'outcome_date', 'age_months']].query("shelter == 'Long Beach'").head(10)


,shelter,dob,outcome_date,age_months
6478,Long Beach,2006-08-21,2021-06-12,180.0
15189,Long Beach,2005-05-20,2017-01-21,142.0
11422,Long Beach,2005-11-25,2021-05-15,188.0
16622,Long Beach,2004-07-15,2017-02-11,153.0
16028,Long Beach,1999-04-01,2023-06-02,294.0
11444,Long Beach,2007-05-26,2021-05-27,170.0
6352,Long Beach,2007-05-12,2018-12-19,141.0
11412,Long Beach,2007-10-27,2020-11-19,159.0
6600,Long Beach,2005-12-20,2021-11-13,193.0
13727,Long Beach,2003-03-15,2017-03-15,170.0


### Group outcome type

In [ ]:
# Define the grouping logic for outcome_type
outcome_group_map = {
    'adoption': 'adopted',
}

# Apply the grouping
df_cat_dog_harmonized['outcome_type_harmonized_grouped'] = (
    df_cat_dog_harmonized['outcome_type_harmonized']
    .map(outcome_group_map)
    .fillna('non-adopted')
)

## AKC Data with Numerical Data Mapping

In [ ]:
df_akc.columns

Index(['dog_breed_name', 'description', 'temperament', 'popularity',
       'min_height', 'max_height', 'min_weight', 'max_weight',
       'min_expectancy', 'max_expectancy', 'group', 'grooming_frequency_value',
       'grooming_frequency_category', 'shedding_value', 'shedding_category',
       'energy_level_value', 'energy_level_category', 'trainability_value',
       'trainability_category', 'demeanor_value', 'demeanor_category'],
      dtype='object')

In [ ]:
#Keep Only Numerical Columns from AKC
akc_numeric_columns = [
    'min_height', 'max_height', 'min_weight', 'max_weight',
    'min_expectancy', 'max_expectancy',
    'grooming_frequency_value', 'shedding_value',
    'energy_level_value', 'trainability_value', 'demeanor_value'
]

In [ ]:
# Ensure breed names are comparable
df_akc['dog_breed_name'] = df_akc['dog_breed_name'].str.upper()
df_cat_dog_harmonized['primary_breed_harmonized'] = df_cat_dog_harmonized['primary_breed_harmonized'].str.upper()

# Subset AKC to only relevant numeric columns
akc_numeric = df_akc[['dog_breed_name'] + akc_numeric_columns].copy()

# Merge
df_cat_dog_harmonized = df_cat_dog_harmonized.merge(
    akc_numeric,
    how='left',
    left_on='primary_breed_harmonized',
    right_on='dog_breed_name'
).drop(columns=['dog_breed_name'])


In [ ]:
df_cat_dog_harmonized[akc_numeric_columns].isna().sum()

,0
min_height,108090
max_height,108090
min_weight,108166
max_weight,108166
min_expectancy,108120
max_expectancy,108120
grooming_frequency_value,108143
shedding_value,108355
energy_level_value,108090
trainability_value,108119


In [ ]:
df_cat_dog_harmonized.columns

Index(['sheltercode', 'outcome_is_dead', 'primary_breed', 'identichipnumber',
       'LastUpdate', 'outcome_year', 'outcome_date', 'Crossing', 'shelter',
       'outcome_type', 'location', 'diedoffshelter', 'geopoint', 'puttosleep',
       'OutcomeCondition', 'deceased_date', 'Jurisdiction', 'animal_name',
       'intake_condition', 'was_outcome_alive', 'primary_color', 'Age',
       'intake_subtype', 'istrial', 'intake_type', 'secondary_color', 'sex',
       'dob', 'longitude', 'outcome_subtype', 'returnedreason',
       'intake_is_dead', 'latitude', 'istransfer', 'return_date', 'animal_id',
       'animal_type', 'intake_date', 'deceasedreason', 'isdoa',
       'intake_reason', 'primary_breed_clean', 'secondary_breed', 'is_mix',
       'intake_date_str', 'has_name', 'stay_length_days', 'Num_returned',
       'Is_returned', 'age_months', 'primary_breed_harmonized',
       'intake_condition_harmonized', 'intake_reason_harmonized',
       'primary_color_harmonized', 'intake_type_harmoniz

In [ ]:
# Identify categorical and numerical columns
categorical_cols = df_cat_dog_harmonized.select_dtypes(include='object').columns
numerical_cols = df_cat_dog_harmonized.select_dtypes(exclude='object').columns

# Fill missin values
df_cat_dog_harmonized[categorical_cols] = df_cat_dog_harmonized[categorical_cols].fillna('unknown')
df_cat_dog_harmonized[numerical_cols] = df_cat_dog_harmonized[numerical_cols].fillna(0)

## Output final dataframe

In [ ]:
output_path = "df_cat_dog_harmonized.csv"
df_cat_dog_harmonized.to_csv(output_path, index=False)